In [ ]:
'''
 * Copyright (c) 2016 Radhamadhab Dalai
 *
 * Permission is hereby granted, free of charge, to any person obtaining a copy
 * of this software and associated documentation files (the "Software"), to deal
 * in the Software without restriction, including without limitation the rights
 * to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the Software is
 * furnished to do so, subject to the following conditions:
 *
 * The above copyright notice and this permission notice shall be included in
 * all copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 * AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 * LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 * OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
 * THE SOFTWARE.
'''

## 2.3 Solving Systems of Linear Equations

In (2.3), we introduced the general form of a system of linear equations:

$$a_{11}x_1 + \cdots + a_{1n}x_n = b_1$$
$$\vdots$$
$$a_{m1}x_1 + \cdots + a_{mn}x_n = b_m \quad (2.37)$$

where $a_{ij} \in \mathbb{R}$ and $b_i \in \mathbb{R}$ are known constants, and $x_j$ are unknowns, for $i = 1, \ldots, m$ and $j = 1, \ldots, n$.

We have seen that matrices can be used to compactly represent systems of linear equations as $Ax = b$, as shown in (2.10). We also defined basic matrix operations like addition and multiplication. In this section, we will focus on solving systems of linear equations and provide an algorithm for finding the inverse of a matrix.

### 2.3.1 Particular and General Solution

Before discussing general methods for solving linear systems, let's look at an example. Consider the system:

$$\begin{bmatrix} 1 & 0 & 8 & -4 \\ 0 & 1 & 2 & 12 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{bmatrix} = \begin{bmatrix} 42 \\ 8 \end{bmatrix} \quad (2.38)$$

This system has two equations and four unknowns, so we expect infinitely many solutions. This system is in a simplified form, where the first two columns consist of a 1 and a 0.

We want to find scalars $x_1, \ldots, x_4$ such that $\sum_{i=1}^4 x_i c_i = b$, where $c_i$ is the $i^{th}$ column of the matrix and $b$ is the right-hand side of (2.38).

A solution can be found by taking 42 times the first column and 8 times the second column:

$$b = \begin{bmatrix} 42 \\ 8 \end{bmatrix} = 42 \begin{bmatrix} 1 \\ 0 \end{bmatrix} + 8 \begin{bmatrix} 0 \\ 1 \end{bmatrix} \quad (2.39)$$

Therefore, a solution is $[42, 8, 0, 0]^\top$. This is called a **particular solution** or **special solution**.

However, this is not the only solution. To capture all other solutions, we need to generate 0 in a non-trivial way using the columns of the matrix. Adding 0 to our special solution does not change it. To do this, we express the third column using the first two columns:

$$\begin{bmatrix} 8 \\ 2 \end{bmatrix} = 8 \begin{bmatrix} 1 \\ 0 \end{bmatrix} + 2 \begin{bmatrix} 0 \\ 1 \end{bmatrix} \quad (2.40)$$

In [1]:
def solve_linear_system_example(A, b):
    """
    Solves the specific linear system example given in the text.

    Args:
        A: The coefficient matrix (list of lists).
        b: The result vector (list).

    Returns:
        A particular solution (list) if found, or None if no solution is found.
    """

    if len(A) != 2 or len(A[0]) != 4 or len(b) != 2:
        print("Error: Input matrices do not match the example system.")
        return None

    if A[0][0] == 1 and A[1][0] == 0 and A[0][1] == 0 and A[1][1] == 1:
        x1 = b[0]
        x2 = b[1]
        x3 = 0
        x4 = 0

        particular_solution = [x1, x2, x3, x4]
        return particular_solution
    else:
        print("Error: Matrix A is not in the expected form.")
        return None

# Example usage
A = [[1, 0, 8, -4], [0, 1, 2, 12]]
b = [42, 8]

particular_solution = solve_linear_system_example(A, b)

if particular_solution:
    print("Particular Solution:", particular_solution)

#Example with a wrong matrix.
A_wrong = [[1,1,1,1],[1,1,1,1]]
b_wrong = [1,2]

particular_solution = solve_linear_system_example(A_wrong, b_wrong)

if particular_solution:
    print("Particular Solution:", particular_solution)

Particular Solution: [42, 8, 0, 0]
Error: Matrix A is not in the expected form.


## Generating the General Solution

We found a particular solution to the system (2.38) as $(x_1, x_2, x_3, x_4) = (42, 8, 0, 0)$. To find the general solution, we need to find non-trivial ways to generate 0 using the columns of the matrix.

We express the third column using the first two columns:

$$0 = 8c_1 + 2c_2 - 1c_3 + 0c_4$$

This gives us the solution $(x_1, x_2, x_3, x_4) = (8, 2, -1, 0)$. Any scaling of this solution by $\lambda_1 \in \mathbb{R}$ produces the 0 vector:

$$\lambda_1 \begin{bmatrix} 1 & 0 & 8 & -4 \\ 0 & 1 & 2 & 12 \end{bmatrix} \begin{bmatrix} 8 \\ 2 \\ -1 \\ 0 \end{bmatrix} = \lambda_1 (8c_1 + 2c_2 - c_3) = 0 \quad (2.41)$$

Similarly, we express the fourth column using the first two columns:

$$0 = -4c_1 + 12c_2 + 0c_3 - 1c_4$$

This gives us another set of non-trivial versions of 0:

$$\lambda_2 \begin{bmatrix} 1 & 0 & 8 & -4 \\ 0 & 1 & 2 & 12 \end{bmatrix} \begin{bmatrix} -4 \\ 12 \\ 0 \\ -1 \end{bmatrix} = \lambda_2 (-4c_1 + 12c_2 - c_4) = 0 \quad (2.42)$$

for any $\lambda_2 \in \mathbb{R}$.

Combining everything, we obtain the general solution of the system (2.38):

$$\left\{ x \in \mathbb{R}^4 : x = \begin{bmatrix} 42 \\ 8 \\ 0 \\ 0 \end{bmatrix} + \lambda_1 \begin{bmatrix} 8 \\ 2 \\ -1 \\ 0 \end{bmatrix} + \lambda_2 \begin{bmatrix} -4 \\ 12 \\ 0 \\ -1 \end{bmatrix}, \lambda_1, \lambda_2 \in \mathbb{R} \right\} \quad (2.43)$$

**Remark:**

The general approach consists of three steps:

1.  Find a particular solution to $Ax = b$.
2.  Find all solutions to $Ax = 0$.
3.  Combine the solutions from steps 1 and 2 to form the general solution.

Neither the general nor the particular solution is unique.

The system of linear equations in the preceding example was easy to solve because the matrix in (2.38) had a convenient form. However, general equation systems are not this simple. Fortunately, Gaussian elimination is a constructive algorithmic way to transform any system of linear equations into this simple form.

### 2.3.2 Elementary Transformations

Gaussian elimination relies on elementary transformations that preserve the solution set while simplifying the equation system.

## 2.3.2 Elementary Transformations

Elementary transformations are crucial for solving systems of linear equations. They preserve the solution set while simplifying the system. These transformations include:

1.  **Exchange of two equations (rows):** Swapping the positions of two rows in the matrix representing the system.
2.  **Multiplication of an equation (row) with a constant $\lambda \in \mathbb{R} \setminus \{0\}$:** Multiplying all elements in a row by a non-zero constant.
3.  **Addition of two equations (rows):** Adding the elements of one row to the corresponding elements of another row.

**Example 2.6:**

We want to find all solutions of the following system of equations, where $a \in \mathbb{R}$:

$$-2x_1 + 4x_2 - 2x_3 - x_4 + 4x_5 = -3$$
$$4x_1 - 8x_2 + 3x_3 - 3x_4 + x_5 = 2$$
$$x_1 - 2x_2 + x_3 - x_4 + x_5 = 0$$
$$x_1 - 2x_2 - 3x_4 + 4x_5 = a \quad (2.44)$$

We begin by converting this system into the compact matrix notation $Ax = b$, and then form the augmented matrix $[A | b]$:

$$\begin{bmatrix} -2 & 4 & -2 & -1 & 4 & | & -3 \\ 4 & -8 & 3 & -3 & 1 & | & 2 \\ 1 & -2 & 1 & -1 & 1 & | & 0 \\ 1 & -2 & 0 & -3 & 4 & | & a \end{bmatrix}$$

We use "$\rightsquigarrow$" to indicate a transformation of the augmented matrix using elementary transformations.

**Step 1: Swap Rows 1 and 3**

$$\begin{bmatrix} 1 & -2 & 1 & -1 & 1 & | & 0 \\ 4 & -8 & 3 & -3 & 1 & | & 2 \\ -2 & 4 & -2 & -1 & 4 & | & -3 \\ 1 & -2 & 0 & -3 & 4 & | & a \end{bmatrix}$$

**Step 2: Apply Row Operations**

$$\begin{bmatrix} 1 & -2 & 1 & -1 & 1 & | & 0 \\ 0 & 0 & -1 & 1 & -3 & | & 2 \\ 0 & 0 & 0 & -3 & 6 & | & -3 \\ 0 & 0 & -1 & -2 & 3 & | & a \end{bmatrix}$$

**Step 3: Further Row Operations**

$$\begin{bmatrix} 1 & -2 & 1 & -1 & 1 & | & 0 \\ 0 & 0 & 1 & -1 & 3 & | & -2 \\ 0 & 0 & 0 & 1 & -2 & | & 1 \\ 0 & 0 & 0 & 0 & 0 & | & a+1 \end{bmatrix}$$

## Row-Echelon Form and Solutions

The resulting augmented matrix is in **row-echelon form (REF)**. Converting this back into the explicit notation with the variables, we get:

<span class="math-block">x\_1 \- 2x\_2 \+ x\_3 \- x\_4 \+ x\_5 \= 0</span>
<span class="math-block">x\_3 \- x\_4 \+ 3x\_5 \= \-2</span>
<span class="math-block">x\_4 \- 2x\_5 \= 1</span>
<span class="math-block">0 \= a \+ 1 \\quad \(2\.45\)</span>

This system can only be solved if <span class="math-inline">a \= \-1</span>.

A **particular solution** is:

<span class="math-block">\\begin\{bmatrix\} x\_1 \\\\ x\_2 \\\\ x\_3 \\\\ x\_4 \\\\ x\_5 \\end\{bmatrix\} \= \\begin\{bmatrix\} 2 \\\\ 0 \\\\ \-1 \\\\ 1 \\\\ 0 \\end\{bmatrix\} \\quad \(2\.46\)</span>

The **general solution**, which captures all possible solutions, is:

<span class="math-block">\\left\\\{ x \\in \\mathbb\{R\}^5 \: <0\>x \= \\begin\{bmatrix\} 2 \\\\ 0 \\\\ \-1 \\\\ 1 \\\\ 0 \\end\{bmatrix\} \+ \\lambda\_1 \\begin\{bmatrix\} 2 \\\\ 1 \\\\ 0 \\\\ 0 \\\\ 0 \\end\{bmatrix\} \+ \\lambda\_2 \\begin\{bmatrix\} 2 \\\\ 0 \\\\ \-1 \\\\ 2 \\\\ 1 \\end\{bmatrix\}, \\lambda\_1, \\lambda\_2</0\> \\in \\mathbb\{R\} \\right\\\} \\quad \(2\.47\)</span>

**Remark (Pivots and Staircase Structure):**

The leading coefficient of a row (first nonzero number from the left) is called the **pivot** and is always strictly to the right of the pivot of the row above it. Therefore, any equation system in row-echelon form has a "staircase" structure.

**Definition 2.6 (Row-Echelon Form):**

A matrix is in row-echelon form if:

1.  All rows containing only zeros are at the bottom of the matrix.
2.  Looking at nonzero rows only, the first nonzero number from the left (the pivot) is always strictly to the right of the pivot of the row above it.

**Remark (Basic and Free Variables):**

The variables corresponding to the pivots in the row-echelon form are called **basic variables**, and the other variables are **free variables**. In (2.45), <span class="math-inline">x\_1, x\_3, x\_4</span> are basic variables, and <span class="math-inline">x\_2, x\_5</span> are free variables.

**Remark (Obtaining a Particular Solution):**

The row-echelon form makes it straightforward to obtain a particular solution.

# Solving a System of Linear Equations in Row-Echelon Form

This (augmented) matrix is in a convenient form, the row-echelon form (REF). Reverting this compact notation back into the explicit notation with the variables we seek, we obtain the following system of equations:

$$
\begin{align}
x_1 - 2x_2 + x_3 - x_4 + x_5 &= 0 \\
x_3 - x_4 + 3x_5 &= -2 \tag{2.45} \\
x_4 - 2x_5 &= 1 \\
0 &= a + 1
\end{align}
$$

Only for \( a = -1 \) can this system be solved. A particular solution is given by:

$$
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5
\end{bmatrix}
=
\begin{bmatrix}
2 \\
0 \\
-1 \\
1 \\
0
\end{bmatrix} \tag{2.46}
$$

The general solution, which captures the set of all possible solutions, is:

$$
\left\{
x \in \mathbb{R} : x =
\begin{bmatrix}
2 \\
0 \\
-1 \\
1 \\
0
\end{bmatrix}
+ \lambda_1
\begin{bmatrix}
2 \\
1 \\
0 \\
0 \\
0
\end{bmatrix}
+ \lambda_2
\begin{bmatrix}
2 \\
0 \\
-1 \\
2 \\
1
\end{bmatrix},
\ \lambda_1, \lambda_2 \in \mathbb{R}
\right\} \tag{2.47}
$$

In the following, we will detail a constructive way to obtain a particular and general solution of a system of linear equations.

## Remark: Pivots and Staircase Structure

The leading coefficient of a row (first nonzero number from the left) is called the *pivot* and is always strictly to the right of the pivot of the row above it. Therefore, any equation system in row-echelon form always has a “staircase” structure.

## Definition 2.6: Row-Echelon Form

A matrix is in *row-echelon form* if:

1. All rows that contain only zeros are at the bottom of the matrix; correspondingly, all rows that contain at least one nonzero element are on top of rows that contain only zeros.
2. Looking at nonzero rows only, the first nonzero number from the left (also called the *pivot* or the *leading coefficient*) is always strictly to the right of the pivot of the row above it.

## Remark: Basic and Free Variables

The variables corresponding to the pivots in the row-echelon form are called *basic variables*, and the other variables are *free variables*. For example, in (2.45), $ x_1, x_3, x_4 $ are basic variables, whereas $ x_2, x_5 $ are free variables.

## Remark: Obtaining a Particular Solution

The row-echelon form makes it straightforward to determine a particular solution by assigning values to the free variables and solving for the basic variables.

In [2]:
import numpy as np

# Define the system of equations in matrix form Ax = b
# For a = -1, the system is consistent
# Variables: x1, x2, x3, x4, x5
# x2 and x5 are free variables

# Coefficient matrix A (for the 3 equations with non-trivial right-hand sides)
A = np.array([
    [1, -2, 1, -1, 1],   # x1 - 2x2 + x3 - x4 + x5 = 0
    [0, 0, 1, -1, 3],    # x3 - x4 + 3x5 = -2
    [0, 0, 0, 1, -2]     # x4 - 2x5 = 1
])

# Right-hand side vector b
b = np.array([0, -2, 1])

# Particular solution provided
x_particular = np.array([2, 0, -1, 1, 0])

# Verify the particular solution
print("Verifying particular solution:")
print("A * x_particular =", A @ x_particular)
print("b =", b)
if np.allclose(A @ x_particular, b):
    print("Particular solution is correct!")
else:
    print("Particular solution is incorrect.")

# General solution: x = x_particular + λ1 * v1 + λ2 * v2
# Free variables are x2 and x5 (indices 1 and 4 in 0-based indexing)
# Basis vectors for the null space from the general solution:
v1 = np.array([2, 1, 0, 0, 0])   # λ1 term
v2 = np.array([2, 0, -1, 2, 1])  # λ2 term

# Test the general solution with sample values for λ1 and λ2
lambda1, lambda2 = 1, -1  # Example values
x_general = x_particular + lambda1 * v1 + lambda2 * v2

print("\nGeneral solution with λ1 = 1, λ2 = -1:")
print("x_general =", x_general)
print("A * x_general =", A @ x_general)
print("b =", b)
if np.allclose(A @ x_general, b):
    print("General solution satisfies the system!")
else:
    print("General solution does not satisfy the system.")

# Function to generate a solution for any λ1, λ2
def get_solution(lambda1, lambda2):
    return x_particular + lambda1 * v1 + lambda2 * v2

# Example usage
print("\nExample solution with λ1 = 3, λ2 = -2:")
x_example = get_solution(3, -2)
print("x =", x_example)
print("A * x =", A @ x_example)

Verifying particular solution:
A * x_particular = [ 0 -2  1]
b = [ 0 -2  1]
Particular solution is correct!

General solution with λ1 = 1, λ2 = -1:
x_general = [ 2  1  0 -1 -1]
A * x_general = [ 0 -2  1]
b = [ 0 -2  1]
General solution satisfies the system!

Example solution with λ1 = 3, λ2 = -2:
x = [ 4  3  1 -3 -2]
A * x = [ 0 -2  1]


In [3]:
# Define the system of equations manually
# A is the coefficient matrix, b is the right-hand side
A = [
    [1, -2, 1, -1, 1],  # x1 - 2x2 + x3 - x4 + x5 = 0
    [0, 0, 1, -1, 3],   # x3 - x4 + 3x5 = -2
    [0, 0, 0, 1, -2]    # x4 - 2x5 = 1
]
b = [0, -2, 1]

# Particular solution provided
x_particular = [2, 0, -1, 1, 0]

# Function to multiply a matrix A by a vector x
def matrix_vector_multiply(A, x):
    result = []
    for row in A:
        sum = 0
        for a, xi in zip(row, x):
            sum += a * xi
        result.append(sum)
    return result

# Function to add vectors
def vector_add(v1, v2):
    return [a + b for a, b in zip(v1, v2)]

# Function to scale a vector by a scalar
def vector_scale(scalar, v):
    return [scalar * vi for vi in v]

# Verify the particular solution
print("Verifying particular solution:")
Ax_particular = matrix_vector_multiply(A, x_particular)
print("A * x_particular =", Ax_particular)
print("b =", b)
if Ax_particular == b:
    print("Particular solution is correct!")
else:
    print("Particular solution is incorrect.")

# General solution: x = x_particular + λ1 * v1 + λ2 * v2
v1 = [2, 1, 0, 0, 0]   # Basis vector for λ1 (x2 free)
v2 = [2, 0, -1, 2, 1]  # Basis vector for λ2 (x5 free)

# Function to compute general solution
def get_general_solution(lambda1, lambda2):
    term1 = vector_scale(lambda1, v1)
    term2 = vector_scale(lambda2, v2)
    return vector_add(x_particular, vector_add(term1, term2))

# Test with sample values for λ1 and λ2
lambda1, lambda2 = 1, -1
x_general = get_general_solution(lambda1, lambda2)

print("\nGeneral solution with λ1 = 1, λ2 = -1:")
print("x_general =", x_general)
Ax_general = matrix_vector_multiply(A, x_general)
print("A * x_general =", Ax_general)
print("b =", b)
if Ax_general == b:
    print("General solution satisfies the system!")
else:
    print("General solution does not satisfy the system.")

# Example with different λ1, λ2
print("\nExample solution with λ1 = 3, λ2 = -2:")
x_example = get_general_solution(3, -2)
print("x =", x_example)
print("A * x =", matrix_vector_multiply(A, x_example))

Verifying particular solution:
A * x_particular = [0, -2, 1]
b = [0, -2, 1]
Particular solution is correct!

General solution with λ1 = 1, λ2 = -1:
x_general = [2, 1, 0, -1, -1]
A * x_general = [0, -2, 1]
b = [0, -2, 1]
General solution satisfies the system!

Example solution with λ1 = 3, λ2 = -2:
x = [4, 3, 1, -3, -2]
A * x = [0, -2, 1]
